In [9]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import os, time, keras

In [4]:
### GLOBALS

X_train_reshaped_path = r"H:\FYP\final\Training_Extracted_MobileNet.npy"
X_test_reshaped_path = r"H:\FYP\final\Testing_Extracted_MobileNet.npy"

y_train_path = r"H:\FYP\final\TrainingLabel.npy"
y_test_path = r"H:\FYP\final\TestingLabel.npy"

if "DenseNet121" in X_train_reshaped_path:
    model_name = "DenseNet121"
elif "MobileNetV2" in X_train_reshaped_path:
    model_name = "MobileNetV2"
elif "EfficientNetB0" in X_train_reshaped_path:
    model_name = "EfficientNetB0"
elif "ShuffleNet" in X_train_reshaped_path:
    model_name = "ShuffleNet"
elif "MobileNet" in X_train_reshaped_path:
    model_name = "MobileNet"

In [5]:
print(f"EXPERIMENT {time.strftime('%y%m%d_%H%M%S')}: {model_name} + RNN\n")

### LOAD FEATURES DATASETS
X_train_reshaped = np.load(X_train_reshaped_path)
X_test_reshaped = np.load(X_test_reshaped_path)

X_train_reshaped.shape, X_test_reshaped.shape

EXPERIMENT 221203_203048: MobileNet + RNN



((3200, 30, 25600), (800, 30, 25600))

In [6]:
y_train = np.load(y_train_path)
y_test = np.load(y_test_path)

y_train.shape, y_test.shape

((3200, 2), (800, 2))

In [30]:
### CREATE LSTM MODEL

model=tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(15, input_shape=(int(X_train_reshaped.shape[1]), int(X_train_reshaped.shape[2])))) # (n_chunks, chunk_size)
model.add(keras.layers.Dense(1024))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(256))
model.add(keras.layers.Activation('sigmoid'))
model.add(keras.layers.Dense(2))
model.add(keras.layers.Activation('softmax'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 15)                1536960   
                                                                 
 dense_56 (Dense)            (None, 1024)              16384     
                                                                 
 activation_56 (Activation)  (None, 1024)              0         
                                                                 
 dense_57 (Dense)            (None, 256)               262400    
                                                                 
 activation_57 (Activation)  (None, 256)               0         
                                                                 
 dense_58 (Dense)            (None, 2)                 514       
                                                                 
 activation_58 (Activation)  (None, 2)               

In [33]:
### TRAIN LSTM MODEL

epoch = 50
batch_size = 30
history=model.fit(x=X_train_reshaped[0:2560], y=y_train[0:2560],
                  epochs=epoch,
                  validation_data=(X_train_reshaped[2560:], y_train[2560:]),
                  callbacks=[tf.keras.callbacks.ModelCheckpoint(f'{model_name}/{model_name}_Checkpoint_4000_{epoch}_{time.strftime("%y%m%d_%H%M%S")}.h5', save_best_only=True)],
                  batch_size=batch_size, verbose=1)

Epoch 1/50
171/171 [==============================] - 51s 296ms/step - loss: 0.2536 - accuracy: 0.5078 - val_loss: 0.2928 - val_accuracy: 0.4719
Epoch 2/50
171/171 [==============================] - 48s 280ms/step - loss: 0.2576 - accuracy: 0.4988 - val_loss: 0.2540 - val_accuracy: 0.5281
Epoch 3/50
171/171 [==============================] - 46s 268ms/step - loss: 0.2562 - accuracy: 0.4844 - val_loss: 0.2502 - val_accuracy: 0.4719
Epoch 4/50
171/171 [==============================] - 47s 278ms/step - loss: 0.2524 - accuracy: 0.5031 - val_loss: 0.2504 - val_accuracy: 0.4719
Epoch 5/50
171/171 [==============================] - 52s 307ms/step - loss: 0.2526 - accuracy: 0.4945 - val_loss: 0.2496 - val_accuracy: 0.5281
Epoch 6/50
171/171 [==============================] - 46s 268ms/step - loss: 0.2562 - accuracy: 0.4949 - val_loss: 0.2566 - val_accuracy: 0.4719
Epoch 7/50
171/171 [==============================] - 45s 266ms/step - loss: 0.2551 - accuracy: 0.5063 - val_loss: 0.2501 - val_ac

KeyboardInterrupt: 

In [ ]:
### EVALUATE LSTM MODEL USING TEST SET
result = model.evaluate(X_test_reshaped, y_test)

In [ ]:
for name, value in zip(model.metrics_names, result):
    print(name, value)

In [ ]:
pred_test_list = []

for i in X_test_reshaped:
    j = i.reshape(1, i.shape[0], int(i.shape[1]))
    predic = model.predict(j)
    pred_test_list.append(predic)

pred_max_list = []
for j in pred_test_list:
    k=np.round(j).astype(int)
    pred_max_list.append(k)

pred_max_list_str = []
decision_list = []
for l in pred_max_list:
    m = l.tobytes().hex()
    if m == '0100000000000000':
        decision_list.append('Non-Violence')
    elif m == '0000000001000000':
        decision_list.append('Violence')
    pred_max_list_str.append(m)


nv_cnt = pred_max_list_str.count('0100000000000000')
v_cnt = pred_max_list_str.count('0000000001000000')

print(f'Non-Violent: {nv_cnt}')
print(f'Violent: {v_cnt}')


df = pd.DataFrame({'Probability':pred_test_list, 'Rounded Predictions': pred_max_list, 'Decision': decision_list})
df.index = np.arange(1, len(df)+1)
df.to_csv(f'{model_name}/{model_name}_RNN_{time.strftime("%y%m%d_%H%M%S")}_Xtest_predictions.csv')

df


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy']) # Draw Accuracy plot
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Training', 'Validation'], loc='lower right')
plt.savefig(f'{model_name}/{model_name}_RNN_Accuracy_{time.strftime("%Y%m%d-%H%M%S")}.jpg') #save .jpg img of Accuracy plot
plt.show()

plt.plot(history.history['loss']) # Draw Loss plot
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.savefig(f'{model_name}/{model_name}_RNN_Loss_{time.strftime("%Y%m%d-%H%M%S")}.jpg') #save .jpg img of Loss plot
plt.show()

In [ ]:
prediction = model.predict(X_test_reshaped)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

sns_true = np.round(prediction).astype(int)
sns_pred = np.round(y_test)

cm = confusion_matrix(sns_true.argmax(axis=1), sns_pred.argmax(axis=1))

sns.heatmap(cm, cmap='Purples', annot=True , fmt = 'g',cbar=True, 
            xticklabels=["Non Violence", "Violence"],
           yticklabels=["Non Violence", "Violence"])
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title(f"Confusion Matrix for {model_name} + RNN\n")
plt.savefig(f'{model_name}/{model_name}_RNN_ConfusionMatrix_{time.strftime("%Y%m%d-%H%M%S")}.jpg')
plt.show()

In [ ]:
print(classification_report(sns_true.argmax(axis=1), sns_pred.argmax(axis=1)))

In [ ]:
model.save(f'{model_name}/Model_{model_name}_RNN_{time.strftime("%Y%m%d-%H%M%S")}.h5')